# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_info = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_info.head(20)


,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_Temp,Wind_Speed
0,0,Henties Bay,0,NaN,1593482903,26,-22.12,14.28,67.66,10.09
1,1,Bethel,1,US,1593482644,82,41.37,-73.41,68.00,4.18
2,2,Saint-François,7,FR,1593482903,87,46.42,3.91,53.01,2.24
3,3,Ushuaia,75,AR,1593482904,64,-54.80,-68.30,33.80,10.29
4,4,Mar del Plata,100,AR,1593482739,76,-38.00,-57.56,52.00,12.75
5,5,Kruisfontein,8,ZA,1593482904,90,-34.00,24.73,50.58,8.66
6,6,Brønnøysund,100,NO,1593482905,93,65.46,12.20,53.60,5.82
7,7,Yumen,100,CN,1593482905,26,40.28,97.20,73.49,18.95
8,8,Albany,60,US,1593482810,86,42.60,-73.97,69.01,5.35
9,9,Dikson,64,RU,1593482905,76,73.51,80.55,49.41,18.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_info[["Latitude", "Longitude"]]
weight = city_info["Humidity"]
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df = city_info.loc[(city_info["Max_Temp"]>70)
                     &(city_info["Max_Temp"]<80)
                     &(city_info["Wind_Speed"]<10)
                     &(city_info["Cloudiness"]==0)
                     &(city_info["Humidity"]<80),:]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_Temp,Wind_Speed
36,36,Marrakesh,0,MA,1593482880,69,31.63,-8.01,73.40,6.93
66,66,Shar,0,KZ,1593482920,26,49.59,81.05,72.70,6.11
76,76,Darhan,0,MN,1593482922,47,49.49,105.92,76.06,6.71
157,157,Korla,0,CN,1593482943,73,41.76,86.15,77.00,3.00
184,184,Follonica,0,IT,1593482949,70,42.93,10.76,73.99,5.91
197,197,Mount Isa,0,AU,1593482952,34,-20.73,139.50,78.80,8.05
225,225,Pouébo,0,NC,1593482959,49,-20.40,164.57,72.64,6.76
279,279,Nola,0,IT,1593482972,78,40.92,14.53,71.60,1.12
353,353,Calvià,0,ES,1593482991,78,39.57,2.51,73.00,3.36
355,355,Corinth,0,GR,1593482991,60,37.94,22.96,75.99,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df["Hotel_Name"]=""
hotel_df

C:\Users\Amber\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_Temp,Wind_Speed,Hotel_Name
36,36,Marrakesh,0,MA,1593482880,69,31.63,-8.01,73.40,6.93,
66,66,Shar,0,KZ,1593482920,26,49.59,81.05,72.70,6.11,
76,76,Darhan,0,MN,1593482922,47,49.49,105.92,76.06,6.71,
157,157,Korla,0,CN,1593482943,73,41.76,86.15,77.00,3.00,
184,184,Follonica,0,IT,1593482949,70,42.93,10.76,73.99,5.91,
197,197,Mount Isa,0,AU,1593482952,34,-20.73,139.50,78.80,8.05,
225,225,Pouébo,0,NC,1593482959,49,-20.40,164.57,72.64,6.76,
279,279,Nola,0,IT,1593482972,78,40.92,14.53,71.60,1.12,
353,353,Calvià,0,ES,1593482991,78,39.57,2.51,73.00,3.36,
355,355,Corinth,0,GR,1593482991,60,37.94,22.96,75.99,1.01,


In [18]:

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
marker_lat=[]
marker_lng = []

for index, row in hotel_df.iterrows():
    try:
        lat = row['Latitude']
        lng = row['Longitude']
        params["location"] = f"{lat},{lng}"
        # run a request using our params dictionary
        hotel_search = requests.get(base_url, params=params)
        hotel_search=hotel_search.json()
        hotel_df.loc[index, "Hotel_Name"] = hotel_search["results"][0]["name"]
        marker_lat.append(hotel_search["results"][0]["geometry"]["location"]["lat"])
        marker_lng.append(hotel_search["results"][0]["geometry"]["location"]["lng"])
    except:
        print(f"No hotel within 5000m of City ID {index}.")
        hotel_df.loc[index, "Hotel_Name"] = "None"
        pass

#dropping cities with no hotel within 5000m
hotel_df = hotel_df.loc[hotel_df["Hotel_Name"]!="None",:]

marker_locations = list(zip(marker_lat,marker_lng))


# Using the template add the hotel marks to the heatmap
fig = gmaps.figure()
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
markers = gmaps.marker_layer(marker_locations,
                             info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig


C:\Users\Amber\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Figure(layout=FigureLayout(height='420px'))

In [19]:
#show completed dataframe
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_Temp,Wind_Speed,Hotel_Name
36,36,Marrakesh,0,MA,1593482880,69,31.63,-8.01,73.40,6.93,Les Jardins de La Koutoubia
76,76,Darhan,0,MN,1593482922,47,49.49,105.92,76.06,6.71,Buudai Hotel
157,157,Korla,0,CN,1593482943,73,41.76,86.15,77.00,3.00,Gulisitan Hotel
184,184,Follonica,0,IT,1593482949,70,42.93,10.76,73.99,5.91,Hotel Parrini
197,197,Mount Isa,0,AU,1593482952,34,-20.73,139.50,78.80,8.05,ibis Styles Mt Isa Verona
279,279,Nola,0,IT,1593482972,78,40.92,14.53,71.60,1.12,Pamaran Hotel
353,353,Calvià,0,ES,1593482991,78,39.57,2.51,73.00,3.36,Castell Son Claret
355,355,Corinth,0,GR,1593482991,60,37.94,22.96,75.99,1.01,Ephira Hotel
483,483,Kutum,0,SD,1593483024,49,14.20,24.67,75.61,4.56,Dibbo areaمنطقة دبو
502,502,Mahon,0,ES,1593483049,76,39.89,4.27,74.89,6.49,ARTIEM Capri
